<a href="https://colab.research.google.com/github/faizdifak/Big-Data/blob/main/Tugas_5_Manipulasi_Dataset_dengan_Apache_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/spscientist/students-performance-in-exams")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: FaizDifak
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/spscientist/students-performance-in-exams


100%|██████████| 8.70k/8.70k [00:00<00:00, 14.1MB/s]

In [ ]:
# Load Dataset
from pyspark.sql import SparkSession

# Buat SparkSession
spark = SparkSession.builder.appName("StudentsPerformance").getOrCreate()

# Load dataset
df = spark.read.csv("/content/students-performance-in-exams/StudentsPerformance.csv", header=True, inferSchema=True)

# Lihat schema dan 5 data awal
df.printSchema()
df.show(5)


root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90

In [ ]:
# Jumlah data
print("Jumlah siswa:", df.count())

Jumlah siswa: 1000


In [ ]:
# Nilai rata-rata per mata pelajaran
df.selectExpr(
    "avg(`math score`) as rata_math",
    "avg(`reading score`) as rata_reading",
    "avg(`writing score`) as rata_writing"
).show()

+---------+------------+------------+
|rata_math|rata_reading|rata_writing|
+---------+------------+------------+
|   66.089|      69.169|      68.054|
+---------+------------+------------+



In [ ]:
# Rata-rata nilai berdasarkan gender
df.groupBy("gender").avg("math score", "reading score", "writing score").show()

+------+------------------+------------------+------------------+
|gender|   avg(math score)|avg(reading score)|avg(writing score)|
+------+------------------+------------------+------------------+
|female|63.633204633204635| 72.60810810810811| 72.46718146718146|
|  male| 68.72821576763485| 65.47302904564316| 63.31120331950208|
+------+------------------+------------------+------------------+



In [ ]:
# Pengaruh kursus persiapan ujian
df.groupBy("test preparation course").avg("math score", "reading score", "writing score").show()

+-----------------------+-----------------+------------------+------------------+
|test preparation course|  avg(math score)|avg(reading score)|avg(writing score)|
+-----------------------+-----------------+------------------+------------------+
|              completed|69.69553072625699| 73.89385474860335| 74.41899441340782|
|                   none| 64.0778816199377| 66.53426791277259| 64.50467289719626|
+-----------------------+-----------------+------------------+------------------+



In [ ]:
# Rata-rata nilai berdasarkan tingkat pendidikan orang tua
df.groupBy("parental level of education") \
  .avg("math score", "reading score", "writing score") \
  .orderBy("avg(math score)", ascending=False).show()

+---------------------------+------------------+------------------+------------------+
|parental level of education|   avg(math score)|avg(reading score)|avg(writing score)|
+---------------------------+------------------+------------------+------------------+
|            master's degree|  69.7457627118644| 75.37288135593221| 75.67796610169492|
|          bachelor's degree| 69.38983050847457|              73.0| 73.38135593220339|
|         associate's degree| 67.88288288288288| 70.92792792792793|  69.8963963963964|
|               some college|  67.1283185840708| 69.46017699115045| 68.84070796460178|
|           some high school|63.497206703910614| 66.93854748603351| 64.88826815642459|
|                high school| 62.13775510204081| 64.70408163265306| 62.44897959183673|
+---------------------------+------------------+------------------+------------------+

